
Ok so this needs to take in a domain, then do:
1) get whois data (call whois)
2) get ip data; get dns data; combine them into a df
3) process the data (artemis_data.process and clean)
4) get dga determination (load dga_predictor and use; or is this a function?)
5) get community prediction (load community predictor and use)
6) 3) encode the data (load encoders and process)
7) get malicious prediction (load malicious predictor and use)


In [ ]:
from src.scripts.artemis import test_entry

from src.scripts.artemis import test_ips, test_entry, test_dns
import json
from src.scripts.artemis_data import process, clean_data, change_ip_data, get_ns_cols, get_email_cols
import whois
import pandas as pd
from src.scripts.get_dns_resolution_data import resolve_dns_records
from src.scripts.get_ip_data import resolve_ip_data
from src.scripts.dga.dga_functions import dga_prediction
from src.scripts.generate_entropy_data import generate_shannon_entropy_score
from joblib import load

In [ ]:
domain = "google.com"
verbose = True

In [ ]:
# w_json = whois.whois(domain)
w_json = test_entry
w_json

In [ ]:
processed_json = process(json.loads(str(w_json)))
processed_json['domain'] = domain
processed_json

In [ ]:
whois_df = pd.DataFrame([processed_json])
whois_df

In [ ]:
#ip_data = {domain: resolve_dns_records(domain, verbose=verbose)}
ip_data = {domain: test_ips}
ip_data

In [ ]:
#dns_data = {domain: resolve_ip_data(ip_data, verbose)}
dns_data = {domain: {'A': {'CC': 'US', 'Org': 'Google LLC'},
            'MX': {'CC': 'US', 'Org': 'Google LLC'}}}
dns_data

In [ ]:
dns_df = pd.DataFrame([change_ip_data(dns_data)])
dns_df

In [ ]:
merged_df = whois_df.merge(dns_df, on='domain')
merged_df['entropy'] = generate_shannon_entropy_score(domain, verbose)
merged_df

In [ ]:
cleaned_df = clean_data(merged_df)
cleaned_df

In [ ]:
country_encoder = load('../models/country_encoder.joblib')
encoder_dict = load('../models/enc_dict.joblib')

In [ ]:
encoded_df = cleaned_df.copy()

In [ ]:
for col in encoder_dict.keys():
    try:
        encoded_df[col] = encoder_dict[col].transform(encoded_df[col])
    except ValueError as ve:
        encoded_df[col] = -1
        print(ve)

In [ ]:
for col in ['country', 'dns_rec_a_cc', 'dns_rec_mx_cc']:
    encoded_df[col] = country_encoder.transform(encoded_df[col])
encoded_df

In [ ]:
for col in get_ns_cols():
    if col in encoded_df.columns.tolist():
        encoded_df.drop(columns=col, inplace=True)
for col in get_email_cols():
    if col in encoded_df.columns.tolist():
        encoded_df.drop(columns=col, inplace=True)
encoded_df = encoded_df.drop(columns=['domain', 'updated_date', 'expiration_date', 'creation_date', 'days_since_creation'])
encoded_df

In [ ]:
community_predictor = load("../models/community_predictor.joblib")
encoded_df['community'] = community_predictor.predict(encoded_df)
encoded_df

In [ ]:
community_df = pd.read_csv('../data/processed/graph_community_features.csv')
community_df = community_df.drop(columns='DomainRecord').drop_duplicates()
community_df

In [ ]:
c_df = community_df[community_df.community==encoded_df.iloc[0].community]
c_df

In [ ]:
cols = c_df.columns.tolist()
for col in cols:
    encoded_df[col] = c_df[col]
encoded_df

In [ ]:
clf = load("../models/rfc.joblib")
clf.predict(encoded_df)

In [ ]:
clf.feature_names_in_

In [ ]:
dir(clf)